In [1]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def showOpencvImage(image, isGray=False):
    fig = plt.figure(figsize=(6, 6))
    plt.imshow(image, cmap = 'gray')
#    plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    plt.show()
    
def readBinaryData(n,SIZE,H,nbytes):

    if nbytes==2:
        d = np.zeros((SIZE,SIZE,H),np.uint16)
    elif nbytes==1:
        d = np.zeros((SIZE,SIZE,H),np.uint8)
    else:
        print('Wrong number of bytes per voxel')
        return
    
    f=open(n,"rb")
    for i in range(0,H):
        for j in range(0,SIZE):
            for k in range(0,SIZE):
                byte = f.read(nbytes)
                if nbytes==2:
                    a = 256*byte[0] + byte[1]
                else:
                    a = byte[0]
                d[j,k,i] = a
    f.close()
    return d




def writeBinaryData(d,n):

    f=open(n,"wb")
    for i in range(0,d.shape[2]):
        for j in range(0,d.shape[0]):
            for k in range(0,d.shape[0]):
                byte = f.write(d[j,k,i])
    f.close()




def components3D(t):
    
    dum = np.zeros(t.shape,dtype=np.uint32)
    label = 0
    
    for x in range(0,t.shape[0]):
        for y in range(0,t.shape[1]):
            for z in range(0,t.shape[2]):
                if t[x,y,z] and dum[x,y,z]==0:
                    label = label+1
                    dum[x,y,z] = label
                    lista = []
                    lista.append((x,y,z))
                    while len(lista):
                        X,Y,Z = lista[0]
                        lista.pop(0)
                        for x1 in range(-1,2):
                            for y1 in range(-1,2):
                                for z1 in range(-1,2):
                                    if X+x1>=0 and X+x1<t.shape[0] and Y+y1>=0 and Y+y1<t.shape[1] and Z+z1>=0 and Z+z1<t.shape[2]:
                                        if t[X+x1,Y+y1,Z+z1] and dum[X+x1,Y+y1,Z+z1]==0:
                                            dum[X+x1,Y+y1,Z+z1] = label
                                            lista.append((X+x1,Y+y1,Z+z1))
    return label,dum


def maxComponent(t):

    n,dum = components3D(t)

    labMax = 0
    masaMax = 0

    for label in range(1,n+1):
        masa = np.sum(dum[dum==label])/label
        if masa>masaMax:
            masaMax = masa
            labMax = label
                        
    t[dum!=labMax] = 0
    return t   

class cluster:
    def __init__(self, label,masa):
        self.label = label
        self.masa = masa


In [3]:
DIR = './'

lab_dir_list = glob.glob(DIR+'Final*.raw')
lab_dir_list.sort()

print(len(lab_dir_list))

for l in lab_dir_list:
    print(l)
    name = l.split('/')[-1].split('.')[0]
    _ , num,SIZE,_,H,B,_ = name.split('_')
   
        
    lab = readBinaryData(l,int(SIZE),int(H),int(B))
        
    for k in range(1,4):
        dum = np.ones(lab.shape,dtype=np.uint8)
        dum[lab!=k]=0
        dum = maxComponent(dum)
        lista = [1,2,3]
        lista.remove(k)
        for n in lista:
            dum[lab==int(n)]=1
        lab = lab*dum
        name = 'Pruned_' + str(num) + '_' + str(lab.shape[0]) + '_' + str(lab.shape[1]) + '_' + str(H) + '_' +'1_.raw'
        writeBinaryData(lab,name)         

        

12
./Final_12_400_400_18_1_.raw
./Final_13_400_400_18_1_.raw
./Final_14_400_400_18_1_.raw
./Final_15_400_400_18_1_.raw
./Final_16_400_400_22_1_.raw
./Final_17_400_400_18_1_.raw
./Final_18_400_400_18_1_.raw
./Final_19_400_400_20_1_.raw
./Final_20_400_400_18_1_.raw
./Final_21_400_400_18_1_.raw
./Final_22_400_400_22_1_.raw
./Final_23_400_400_20_1_.raw


In [4]:
DIR = './'

lab_dir_list = glob.glob(DIR+'Pruned*.raw')
lab_dir_list.sort()

print(len(lab_dir_list))
for l in lab_dir_list:
    print(l)
    name_l = l.split('/')[-1].split('.')[0]
    _ , num_l,SIZE_l,_,H_l,B_l,_ = name_l.split('_')
    
    lab = readBinaryData(l,int(SIZE_l),int(H_l),int(B_l))
    print(np.sum(lab[lab==1]))
    print(np.sum(lab[lab==2]))
    print(np.sum(lab[lab==3]))

30
./Pruned_10_400_400_18_1_.raw
34496
163130
98571
./Pruned_11_400_400_18_1_.raw
34494
177766
117513
./Pruned_1_400_400_18_1_.raw
37221
163360
85734
./Pruned_24_400_400_20_1_.raw
47554
153544
139632
./Pruned_25_400_400_21_1_.raw
43533
126214
126120
./Pruned_26_400_400_18_1_.raw
49934
197730
141003
./Pruned_27_400_400_20_1_.raw
26657
121338
67584
./Pruned_28_400_400_18_1_.raw
38004
73160
135936
./Pruned_29_400_400_18_1_.raw
39455
81148
129069
./Pruned_2_400_400_18_1_.raw
28710
104268
82491
./Pruned_30_400_400_22_1_.raw
28501
129300
96168
./Pruned_31_400_400_24_1_.raw
64345
209492
159651
./Pruned_32_400_400_18_1_.raw
32427
164714
96366
./Pruned_33_400_400_18_1_.raw
32295
151494
92841
./Pruned_34_400_400_18_1_.raw
34618
146888
96987
./Pruned_35_400_400_20_1_.raw
27803
111922
84120
./Pruned_36_400_400_18_1_.raw
37271
147172
89394
./Pruned_37_400_400_23_1_.raw
42549
127418
116541
./Pruned_38_400_400_22_1_.raw
49327
196924
135078
./Pruned_39_400_400_22_1_.raw
38136
118764
92427
./Pruned_3_4